In [1]:
import h5py
# import numpy as np
import torch
from einops import rearrange, repeat

In [2]:
data_num = 100
k=4
input_file = f"../data/ns_contextual/ns_random_forces_bottom{data_num}_mu.h5"
output_file = f"../data/ns_contextual/ns_random_forces_bottom{data_num}_mu_k_{k}.h5"

In [3]:
# output_file = f"../data/ns_contextual/ns_random_forces_few_shot_bottom{data_num})_mu.h5"
with h5py.File(input_file, 'r') as f_in:
    u0 = torch.tensor(f_in['u'])
    f0 = torch.tensor(f_in['f'])
    mu0 = torch.tensor(f_in['mu'])
    dtype = f_in['u'].dtype
    f_in.close()

B = u0.shape[0]
L = u0.shape[-1]
L1 = L // k
u1_shape = [k] + list(u0.shape); u1_shape[-1] = L1
# f1_shape = [k] + list(f0.shape)
# mu1_shape = [k] + list(mu0.shape)
u1 = torch.zeros(u1_shape)
# f1 = np.zeros(f1_shape)
# mu1 = np.zeros(mu1_shape)
f1 = repeat(f0, 'a b c -> k a b c', k=k) * (k*k)
mu1 = repeat(mu0, 'a -> k a', k=k) * k
for i in range(k):
    u1[i,...] = k * u0[..., i:k*L1+i:k]


/tmp/ipykernel_116659/619507543.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  u0 = torch.tensor(f_in['u'])
/home/yichen/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
u1 = u1.view(-1,*u1_shape[2:])
f1 = f1.view(-1,*f1.shape[2:])
mu1 = mu1.view(-1)

In [5]:
f1.shape

torch.Size([400, 256, 256])

In [10]:
torch.sum(torch.abs(u1[0, :, :, 0]-u1[100, :, :, 0]))

tensor(2585.6270)

In [12]:
torch.sum(torch.abs(u0[0, :, :, 0]-u0[0, :, :, 1]))*4

tensor(2585.6270)

In [5]:
with h5py.File(output_file, 'w') as f_out:
    u1_out = f_out.create_dataset('u', data=u1, dtype=dtype)
    f1_out = f_out.create_dataset('f', data=f1, dtype=dtype)
    mu1_out = f_out.create_dataset('mu', data=mu1, dtype=dtype)
    f_out.close()

In [13]:
from neuralop.datasets.autoregressive_dataset import load_autoregressive_traintestsplit_v3

train_loader, test_loaders = load_autoregressive_traintestsplit_v3(
        '../data/ns_contextual/ns_random_forces_top100_mu.h5',
        100, 1,
        64, 64,
        4, 4,
        10,
        predict_feature='u',
    )

/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [26]:
train_loader

In [27]:
from scripts.ns_contextual_trainer import gen_similar_dataloaders_dt_divided_p
dict_p_loaders = gen_similar_dataloaders_dt_divided_p(train_loader, scaling_ps=[4])
ploader = dict_p_loaders['dt/=p_4']

In [41]:
ploader.dataset.get_item(0)

{'y': tensor([[ 0.0493,  0.0713,  0.0872,  ..., -0.0151,  0.0092,  0.0286],
         [ 0.0482,  0.0708,  0.0856,  ..., -0.0131,  0.0094,  0.0271],
         [ 0.0679,  0.0873,  0.0999,  ...,  0.0103,  0.0324,  0.0493],
         ...,
         [ 0.0894,  0.1277,  0.1593,  ..., -0.0236,  0.0113,  0.0491],
         [ 0.0848,  0.1144,  0.1377,  ..., -0.0072,  0.0234,  0.0538],
         [ 0.0680,  0.0914,  0.1090,  ..., -0.0080,  0.0195,  0.0441]]),
 'f': tensor([[0.2379, 0.2601, 0.2754,  ..., 0.1374, 0.1756, 0.2095],
         [0.2567, 0.2755, 0.2874,  ..., 0.1647, 0.2005, 0.2315],
         [0.2707, 0.2861, 0.2946,  ..., 0.1881, 0.2210, 0.2488],
         ...,
         [0.1570, 0.1876, 0.2123,  ..., 0.0387, 0.0813, 0.1211],
         [0.1874, 0.2156, 0.2374,  ..., 0.0735, 0.1152, 0.1536],
         [0.2146, 0.2399, 0.2586,  ..., 0.1067, 0.1470, 0.1833]]),
 'mu': tensor(9.8323e-05),
 'x': tensor([[-0.2266, -0.2142, -0.1996,  ..., -0.1818, -0.1834, -0.2138],
         [-0.2078, -0.2092, -0.1964,  .

: 

In [32]:
u1[0, :, :, 1]

tensor([[0.8234, 0.8481, 0.8755,  ..., 0.7241, 0.7616, 0.7953],
        [0.8763, 0.8992, 0.9277,  ..., 0.7813, 0.8199, 0.8516],
        [0.9236, 0.9470, 0.9772,  ..., 0.8348, 0.8723, 0.9008],
        ...,
        [0.6419, 0.6603, 0.6792,  ..., 0.5687, 0.5938, 0.6194],
        [0.7033, 0.7258, 0.7492,  ..., 0.6184, 0.6480, 0.6775],
        [0.7654, 0.7901, 0.8158,  ..., 0.6700, 0.7039, 0.7367]])